## Ejercicio 2

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

__IMPORTANTE__: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.

In [1]:
import string
import re 
import nltk
import urllib.request
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

# BeautifulSoup y lxml para analizar y navegar por estructuras HTML y XML
import lxml
import bs4 as bs

# Descargar recursos necesarios de NLTK
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Argentina')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Eliminar todas las etiquetas <sup> y su contenido
for sup_tag in article_html.find_all('sup'):
    sup_tag.decompose()

# Encontrar todos los párrafos y unirlos en un solo texto
article_paragraphs = article_html.find_all('p')
article_text = ''.join(para.text for para in article_paragraphs)

article_text = ''

for para in article_paragraphs:
    article_text += para.text

In [24]:
print("Cantidad de caracteres en el articulo:", len(article_text))

Cantidad de caracteres en el articulo: 190770


### Preprocesamiento

In [25]:
def preprocess_text(text):
    # Quitar tildes y otros caracteres diacríticos
    text = ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))

    # Remover caracteres especiales (manteniendo algunos signos de puntuación importantes)
    pattern = r'[^a-zA-Z0-9.,!?/:;\"\'\s]'
    text = re.sub(pattern, '', text)

    # Sustituir más de un caracter de espacio, salto de línea o tabulación por un solo espacio
    text = re.sub(r'\s+', ' ', text)

    # Convertir a minúsculas
    text = text.lower()

    return text

# Ahora aplicamos la función al texto del artículo
preprocessed_article_text = preprocess_text(article_text)

In [26]:
print("Cantidad de caracteres en el articulo:", len(preprocessed_article_text))

Cantidad de caracteres en el articulo: 188994


### Dividir el texto en oraciones y palabras

In [27]:
# Dividir el texto en oraciones
sentences = nltk.sent_tokenize(preprocessed_article_text, language='spanish')

# Dividir cada sentencia en palabras
words_in_sentences = nltk.word_tokenize(preprocessed_article_text, language='spanish')

print("Sentencias:", len(sentences))
print("Palabras en las sentencias:", len(words_in_sentences))

Sentencias: 924
Palabras en las sentencias: 33952


In [28]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def get_processed_text(document):
    return nltk.word_tokenize(preprocess_text(document))

### Stopwords

In [29]:
# Acceder a las stopwords en español
spanish_stopwords = stopwords.words('spanish')

# Ejemplo de stopwords en español
print(spanish_stopwords[:10])  # Imprime las primeras 10 stopwords para ejemplificar

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']


In [30]:
preprocessed_stopwords = []

for w in spanish_stopwords:
    preprocessed_word = preprocess_text(w)
    preprocessed_stopwords.append(preprocessed_word)

print(preprocessed_stopwords[:10])  # Imprime las primeras 10 stopwords preprocesadas

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']


In [74]:
def generate_response(user_input, corpus):
    response = ''

    # Agrega la entrada del usuario al corpus
    corpus.append(user_input)

    # Inicializa el vectorizador TFIDF con tokens lematizados
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=preprocessed_stopwords)

    # Vectoriza el corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcula similitud coseno con todos los documentos
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    
    # Encuentra el índice del documento más similar
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    # Imprime el valor de similitud coseno
    print('cosine similarity value: ', vector_matched)

    # Determina la respuesta basada en la similitud
    if vector_matched < 0.2:
        response = "Disculpame, no puedo responder a esa pregunta"
    else:
        response = corpus[similar_sentence_number]
    
    # Elimina la entrada del usuario del corpus
    corpus.remove(user_input)
    return response

In [75]:
# Ejemplo
user_input = "cual es la lengua predominante en argentina"
generate_response(user_input, sentences)

cosine similarity value:  0.4836918974330471


'actualmente, el castellano es la lengua predominante, entendido y hablado como primera o segunda lengua por casi toda la poblacion de la argentina, que segun el censo de 2022 llega a 47,2 millones de habitantes.'

In [76]:
user_input = "cual es el IDH de argentina"
generate_response(user_input, sentences)

cosine similarity value:  0.4629284275764496


'de acuerdo con el informe sobre desarrollo humano del programa de las naciones unidas para el desarrollo de 2018, la argentina tiene un indice de desarrollo humano idh de 0,825. a nivel mundial, se situa en el puesto 47 dentro de los 189 estados que participan en la clasificacion, encasillado como un pais de idh muy alto que junto con chile y uruguay son los unicos paises de america latina que se encuentran en este nivel de idh.'

In [77]:
user_input = "como es la economia argentina"
generate_response(user_input, sentences)

cosine similarity value:  0.43562585331374387


'la economia argentina es la segunda mas desarrollada e importante en sudamerica.'

In [78]:
user_input = "como es la cultura argentina"
generate_response(user_input, sentences)

cosine similarity value:  0.44543156095093006


'el escritor argentino ernesto sabato ha reflexionado sobre la naturaleza de la cultura argentina del siguiente modo: la cultura argentina tiene como origen la mezcla de otras que se encontraron durante los anos de las inmigraciones.'